# Metric Learning for Clustering Discrete Sequences

* https://stackoverflow.com/questions/38260113/implementing-contrastive-loss-and-triplet-loss-in-tensorflow
* http://scikit-learn.org/stable/modules/manifold.html

# Preparation:
* define experiment X in config/all_experiments.py
* execute 010_generate_vocabulary.py -en X
* execute 020_generate_training_sequences.py -en X
* execute 025_extract_signatures.py -en X

# Papers

* [1] FaceNet https://www.cv-foundation.org/openaccess/content_cvpr_2015/papers/Schroff_FaceNet_A_Unified_2015_CVPR_paper.pdf
* [2] Siamese Network: http://yann.lecun.com/exdb/publis/pdf/chopra-05.pdf
* [3] Triplet Network: https://www.cv-foundation.org/openaccess/content_cvpr_2014/papers/Wang_Learning_Fine-grained_Image_2014_CVPR_paper.pdf




# 0 Setup notebook

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Imports

In [2]:
import tensorflow as tf
assert tf.__version__.startswith("1.4") # the version we used
import numpy as np
import os
from os.path import join as jp
import logging 
import library.helpers as h
import library.tensorflow_helpers as tfh
import time
from library.vocabulary import *
from tensorflow.contrib.tensorboard.plugins import projector # for visualizing embeddings
import re
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import pairwise_distances
np.set_printoptions(precision=3, suppress=True)


import matplotlib # plotting stuff
matplotlib.use('Agg') # for displaying plots in console without display

## Configurations

In [3]:
load_from_file=True
# fix training
RANDOM_SEED = 0 
# configure numpy 
np.set_printoptions(precision=3, suppress=True)
np.random.seed(RANDOM_SEED)

# configure tensorflow
tf.set_random_seed(RANDOM_SEED)

# configure logger
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# configure ipython display
def show(img_file):
    try: # only works in ipython notebook
        display(Image(filename=img_file))
    except:
        pass

# 0.4 Create directories

In [4]:
LOG_NAME = "bgl2" # [unix_forensic, bgl2, spirit2, synthetic_10, synthetic_reverse_10, newsgroup20_200]
MODEL_NAME = "basline-jaccard"
DATA_DIR = "data"
RESULTS_DIR = jp("results", LOG_NAME, MODEL_NAME)
h.create_dir(RESULTS_DIR)
VIZUALIZATIONS_DIR = "visualizations"
INPUTS_DIR = jp(DATA_DIR, "inputs")

# result files
SHARD_SIZE = 7900 # spirit2 6500
RESULTS_FILE=jp(RESULTS_DIR, "%0.5d-%0.2f-results.csv")

ENCODER_INPUTS_PATH = jp(DATA_DIR, "encoder_inputs", "%s.idx"%LOG_NAME)
ENC_SEQUENCE_LENGTH_PATH = jp(DATA_DIR, "sequence_lengths", "%s_enc.idx"%LOG_NAME)

RAW_LOG = jp(DATA_DIR, "raw", "%s.log"%LOG_NAME)
LOGLINES = np.array([l[:-1] for l in list(open(RAW_LOG))])

SIGNATURE_FILE =jp(DATA_DIR, "signatures","%s.sig"%LOG_NAME)
SIGNATURES = np.array(list(open(SIGNATURE_FILE)))

h.create_dir(DATA_DIR)  # power traces go here
h.create_dir(INPUTS_DIR)
h.create_dir(VIZUALIZATIONS_DIR) # charts we generate
h.create_dir(RESULTS_DIR)

h.create_dir("graphs") 

TAG_NUM = -1 # set >1 to use a specific tag

if TAG_NUM < 0:
    TAG = "%0.3d"%(len(os.listdir("graphs"))+1)
    DO_TRAINING = True
else:
    TAG = "%0.3d"%(TAG_NUM)
    DO_TRAINING = False

GRAPH_DIR = jp("graphs", "%s-%s"%(MODEL_NAME, TAG))
h.create_dir(GRAPH_DIR) # store tensorflow calc graph here 



INFO:library.helpers:Created directory: graphs/basline-jaccard-156


# 1 Hyper Parameters 

In [5]:
BATCH_SIZE = 100 # 73 62 
NUM_EPOCHS = 20
MAX_GRADIENT_NORM = 0.5
STATE_SIZE = 128 #  32
test_fraction=0.1

NUM_LSTM_LAYERS = 1
ALPHA = 1.0 # distance margin 
DTYPE = tf.float32 # datatype for network parameters

NUM_SEQUENCES = len(SIGNATURES)

LEARNING_RATE_DECAY_FACTOR = 0.95
MAX_ENC_SEQ_LENGTH =  max([int(s) for s in list(open(ENC_SEQUENCE_LENGTH_PATH,"r"))])
VOCABULARY = Vocabulary.load(LOG_NAME, "")


TF_LEARNING_RATE = tf.Variable(0.0001, trainable=False, name="Learning_rate") # alpha of our training step
TF_KEEP_PROBABILTIY = tf.Variable(1.0, trainable=False, name="Dropout_keep_probability") # can be added to feeddict
TF_GLOBAL_STEP = tf.Variable(0, trainable=False, name="Global_step") # keeps track of the current training step

logger.info("Max. Encoder Sequence Length: %s"%MAX_ENC_SEQ_LENGTH)

INFO:__main__:Max. Encoder Sequence Length: 176


In [6]:
VOCABULARY.size()

101872

In [7]:
def roundup(x, to=100):
    return int(math.ceil(x / to)) * to

def rounddown(x,to=100):
    return int(math.floor(x / to)) * to

# 2 Data

* Sequence of tokens $T$ 
* We build a vocabulary, which is a map of each unique item in the vocabulary to an integer

* To generate your training / test sequences, execute scripts: 010, 020, and 025. 




# 2.1 Parse data to Memmap


In [8]:
def parse_input_line(line, max_seq_length):
    split_line = line[:-1].split(" ") # cut \n at the end
    
    split_line_ints = [int(sl) for sl in split_line if len(sl.strip())>0] # pad sequence with zeros
    padding  = [0] * (max_seq_length - len(split_line_ints))
    padded_line_ints = split_line_ints +  padding
    return np.array(padded_line_ints)

def parse_input_file(input_file, output_file,  max_seq_length, force_regeneration=False, dtype="int32"):
    output_path = jp(INPUTS_DIR, output_file)
    if not h.file_exists(output_path) or force_regeneration:
        fp = np.memmap(output_path, dtype=dtype, mode='w+', shape=(NUM_SEQUENCES,max_seq_length))
        # save inputs to memmap
        for line_id, line in enumerate(list(open(input_file,"r"))):
            #print(line, parse_input_line(line, max_seq_length))
            fp[line_id,:]= parse_input_line(line, max_seq_length)
        
    else:
        logger.info(output_path +" already exists, delete it for regeneration.")
        fp = np.memmap(output_path, dtype=dtype, mode='r', shape=(NUM_SEQUENCES,max_seq_length))
    return fp


# load memmaps for seqlength (enc,dec) and (x_enc x_dec y_dec )
ENCODER_INPUTS  = parse_input_file(ENCODER_INPUTS_PATH, "enc_input-%s.mm"%LOG_NAME ,  MAX_ENC_SEQ_LENGTH, force_regeneration=True)
ENCODER_SEQLENGTH = np.array([int(s) for s in list(open(ENC_SEQUENCE_LENGTH_PATH,"r"))])
SIGNATURE_FILE =jp(DATA_DIR, "signatures","%s.sig"%LOG_NAME)
SIGNATURES = np.array(list(open(SIGNATURE_FILE)))

print("Encoder inputs shape: ",ENCODER_INPUTS.shape)
print("Encoder inputs(tok): ", VOCABULARY.index_seq_to_line(ENCODER_INPUTS[0:1,:].flatten()))
print("Encoder inputs(int):", ENCODER_INPUTS[0:1,:], "Length:",  ENCODER_SEQLENGTH[0])
print("")

Encoder inputs shape:  (474796, 176)
Encoder inputs(tok):  - time _ stamp short _ date node _ id _ 01 date _ time node _ id _ 01 ras kernel info instruction cache parity error corrected PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_TOKEN PAD_

# Pairwise Jaccard distance

In [9]:
def jaccard_distance(s1, s2): 
    len_union = np.count_nonzero(np.union1d(s1,s2))
    len_intersect = np.count_nonzero(np.intersect1d(s1,s2, ))
    if len_union==0:
        return 1.0
    else:
        return 1- (len_intersect / len_union)

In [10]:
np.random.seed(0)
random_permutation = np.random.permutation(ENCODER_INPUTS.shape[0])

TEST_START_INDEX = roundup(int(NUM_SEQUENCES*(1-test_fraction)))
TEST_END_INDEX = rounddown(NUM_SEQUENCES)
# LOGLINES = np.array(list(open(jp(DATA_DIR, "raw", "%s.log"%LOG_NAME) )))
SIGNATURES=np.array([int(s) for s in SIGNATURES])

if test_fraction>0: # if a test / train fraction is defined 
    TEST_INPUTS = ENCODER_INPUTS[random_permutation][TEST_START_INDEX:TEST_END_INDEX]
    TEST_LABELS = SIGNATURES[random_permutation][TEST_START_INDEX:TEST_END_INDEX]
    # LOGLINES_TEST = LOGLINES[random_permutation][TEST_START_INDEX:TEST_END_INDEX] 
else: # otherwise use whole dataset for test / train
    0/0 
    pass 

   
print("Using %0.2f test fraction (0.0=all)"%test_fraction)
print("num test sequences:", TEST_INPUTS.shape)


Using 0.10 test fraction (0.0=all)
num test sequences: (47300, 176)


# Validation rate (d)

# generate final results 

In [11]:
def evaluate_shard(out_csv_name, pw_ji, labels_x, labels_y, d = 0.00, d_step = 0.005, d_max=1.0):
    
    h.save_to_csv(data_rows=[[
        "Distance Threshhold",
        "True Positives", 
        "False Positives", 
        "True Negative", 
        "False Negative", 
        "Num True Same", 
        "Num True Diff", 
    ]], outfile_name=out_csv_name, mode="w")
    
    
    # calculate true accepts / false accepts based on labels
    n_labels = len(labels_x)
    tl_row = np.repeat( np.array(labels_x).reshape((n_labels,1)), n_labels, axis=1 )
    tl_col = np.repeat( np.array(labels_y).reshape((1,n_labels)), n_labels, axis=0 ) 
    p_same = np.equal(tl_row, tl_col).astype("int8")
    p_diff = np.not_equal(tl_row, tl_col).astype("int8")
    num_true_same = p_same.sum()
    num_true_diff = p_diff.sum()
    
    while True:
        calc_same = np.zeros((n_labels, n_labels))
        calc_same[np.where(pw_ji<=d)]=1
        
        tp = np.sum(np.logical_and(calc_same, p_same))
        fp = np.sum(np.logical_and(calc_same, np.logical_not(p_same)))
        tn = np.sum(np.logical_and(np.logical_not(calc_same), np.logical_not(p_same)))
        fn = np.sum(np.logical_and(np.logical_not(calc_same), p_same))
        
        h.save_to_csv(data_rows=[[d, tp, fp, tn, fn,num_true_same,num_true_diff]], outfile_name=out_csv_name, mode="a")
        
        d+=d_step
        if d>d_max:
            break

def evaluate_all_shards(inputs, labels, shard_size,shard_indizes,  results_fn, d_start=0.0, d_step=0.005, d_max=1.0 ):
    num_test_examples = inputs.shape[0]
    times = []
    for i, shard_index in enumerate(shard_indizes):
        s = time.time()
        shard_x, shard_y = shard_index
        out_csv_name = results_fn+"_%0.2d-%0.2d"%(shard_x, shard_y)
        if os.path.exists(out_csv_name):
            shard_data = h.load_from_csv(out_csv_name)
            if len(shard_data) == len(np.arange(d_start, d_max, d_step))+1: # data was completely loaded, don't need to regenerate this shard
                print("Shard %i exists and is complete, skipping"%i)
                continue
        print("Current shard", shard_index, "%i/%i"%(i, len(shard_indizes)))
        start_index_x = shard_x*shard_size
        start_index_y = shard_y*shard_size
        end_index_x = min((shard_x+1)*shard_size, num_test_examples)
        end_index_y = min((shard_y+1)*shard_size, num_test_examples)

        # calcualte pairwise distances
        shard_inputs_x = inputs[start_index_x:end_index_x,:]
        shard_labels_x = labels[start_index_x:end_index_x]

        shard_inputs_y = inputs[start_index_y:end_index_y,:]
        shard_labels_y = labels[start_index_y:end_index_y]

        pw_ji = pairwise_distances(shard_inputs_x,shard_inputs_y, metric=jaccard_distance, n_jobs=8) 

        # evaluate pairwise distances 
        
        evaluate_shard(out_csv_name, pw_ji, shard_labels_x, shard_labels_y, d=d_start,  d_step = d_step, d_max=d_max)
        e=time.time()
        times.append( (e-s)/60 )
        print("Avg time in min for shard: %0.2f  "%np.mean(times))
            
def run_evaluation(inputs, labels, shard_size, results_fn, d_start=0.0, d_step=0.005, d_max=1.0):
    results_fn = results_fn%(shard_size, test_fraction)
    
    num_test_examples = inputs.shape[0]
    num_x = inputs.shape[0]//shard_size
    if not num_test_examples%shard_size==0 :# need to be a square matrix
        print("Allowed shard sizes")
        for i in range(100, num_test_examples):
            if num_test_examples%i==0:
                print(i)
        0/0
    shard_indizes = list(itertools.product(range(num_x),repeat=2))
    num_shards = len(shard_indizes)
    num_distances = len(list(np.arange(d_start,d_max,d_step)))
    num_metrics = 7 
    
    evaluate_all_shards(inputs, labels, shard_size, shard_indizes, results_fn, d_start, d_step, d_max )
    
    all_data = np.ndarray(shape=(num_shards, num_distances, num_metrics), dtype="float32")

    for i, shard_index in enumerate(shard_indizes):
        # load shard
        shard_x, shard_y = shard_index
        out_csv_name = results_fn+"_%0.2d-%0.2d"%(shard_x, shard_y)
        shard_data = h.load_from_csv(out_csv_name)
        shard_data = shard_data[1:] # cut header row 
        all_data[i] = np.array(shard_data)


    final_data  = np.ndarray(shape=(num_distances, 10), dtype="float32")

    final_data[:,0] = all_data[0,:,0] # all distances (are same over all shards)

    final_data[:,1] = all_data.sum(axis=0)[:,1] # True Positives
    final_data[:,2] = all_data.sum(axis=0)[:,2] # False Positives
    final_data[:,3] = all_data.sum(axis=0)[:,3] # True Negatives
    final_data[:,4] = all_data.sum(axis=0)[:,4] # False Negatives
    final_data[:,5] = all_data.sum(axis=0)[:,5] # Num true same (are same over all shards)
    final_data[:,6] = all_data.sum(axis=0)[:,6] # Num true diff  (are same over all shards)

    final_data[:,7] = final_data[:,1]/final_data[:,5] # validation rate 
    final_data[:,8] = final_data[:,2]/final_data[:,6] # false acceptance rate  

    final_data[:,9] = (final_data[:,1] + final_data[:,3]) / (final_data[:,1:1+4].sum(axis=1)) 

    
    h.save_to_csv(data_rows=[[
            "Distance Threshhold",
            "True Positives", 
            "False Positives", 
            "True Negative", 
            "False Negative", 
            "Num true same", 
            "Num true diff", 
            "Validation Rate",
            "False Acceptance Rate",
            "Accuracy"
        ]], outfile_name=results_fn, mode="w", convert_float=False)
    h.save_to_csv(data_rows=final_data, outfile_name=results_fn, mode="a", convert_float=True)

    logger.info("Evaluation done, saved to '%s'"%results_fn)
    return final_data

In [12]:
shards_9460 = run_evaluation(inputs=TEST_INPUTS, labels=TEST_LABELS, shard_size=9460, results_fn=RESULTS_FILE)

Current shard (0, 0) 0/25
Avg time in min for shard: 17.58  
Current shard (0, 1) 1/25
Avg time in min for shard: 17.68  
Current shard (0, 2) 2/25
Avg time in min for shard: 17.70  
Current shard (0, 3) 3/25
Avg time in min for shard: 17.71  
Current shard (0, 4) 4/25
Avg time in min for shard: 17.74  
Current shard (1, 0) 5/25
Avg time in min for shard: 17.75  
Current shard (1, 1) 6/25
Avg time in min for shard: 17.75  
Current shard (1, 2) 7/25
Avg time in min for shard: 17.77  
Current shard (1, 3) 8/25
Avg time in min for shard: 17.80  
Current shard (1, 4) 9/25
Avg time in min for shard: 17.82  
Current shard (2, 0) 10/25
Avg time in min for shard: 17.84  
Current shard (2, 1) 11/25
Avg time in min for shard: 17.86  
Current shard (2, 2) 12/25
Avg time in min for shard: 17.88  
Current shard (2, 3) 13/25
Avg time in min for shard: 17.88  
Current shard (2, 4) 14/25
Avg time in min for shard: 17.90  
Current shard (3, 0) 15/25
Avg time in min for shard: 17.90  
Current shard (3, 

INFO:__main__:Evaluation done, saved to 'results/bgl2/basline-jaccard/09460-0.10-results.csv'


Avg time in min for shard: 17.86  
